In [12]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# -----------------------------------------------
# 1. Preprocessing Class
# -----------------------------------------------
class Preprocessing:
    def __init__(self, text_path):
        self.text_path = text_path
        self.text = ''
        self.char2idx = {}
        self.idx2char = {}
        self.vocab_size = 0
        self.encoded = None
        self.one_hot = None

    def read_text(self):
        with open(self.text_path, 'r', encoding='utf-8') as f:
            self.text = f.read()
        return self.text

    def one_hot_encode(self, encoded, vocab_size):
        return np.eye(vocab_size)[encoded]

    def processing(self):
        text = self.read_text()
        cleaned_text = re.sub(r'[^a-zA-Z\s\.\?,]', '', text).lower()
        unique_chars = sorted(set(cleaned_text))
        self.vocab_size = len(unique_chars)
        self.char2idx = {ch: i for i, ch in enumerate(unique_chars)}
        self.idx2char = {i: ch for i, ch in enumerate(unique_chars)}
        self.encoded = np.array([self.char2idx[ch] for ch in cleaned_text])
        self.one_hot = self.one_hot_encode(self.encoded, self.vocab_size)
        return self.encoded

# -----------------------------------------------
# 2. Custom LSTM Model
# -----------------------------------------------
class CustomLSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        emb = self.embedding(x)
        lstm_out, _ = self.lstm(emb)
        last_hidden = lstm_out[:, -1, :]
        return self.fc(last_hidden)

# -----------------------------------------------
# 3. ModAL-style Wrapper
# -----------------------------------------------
class CustomModALWrapper:
    def __init__(self, model, device=None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)

    def predict(self, X):
        self.model.eval()
        Xb = torch.LongTensor(X).to(self.device)
        with torch.no_grad():
            logits = self.model(Xb)
        return logits.argmax(dim=1).cpu().numpy()

    def predict_proba(self, X):
        self.model.eval()
        Xb = torch.LongTensor(X).to(self.device)
        with torch.no_grad():
            logits = self.model(Xb)
            probs = torch.softmax(logits, dim=1)
        return probs.cpu().numpy()

    def fit(self, X, y, epochs=3, batch_size=32, lr=2e-5):
        Xt = torch.LongTensor(X)
        yt = torch.LongTensor(y)
        ds = TensorDataset(Xt, yt)
        loader = DataLoader(ds, batch_size=batch_size, shuffle=True)
        opt = optim.AdamW(self.model.parameters(), lr=lr)
        crit = nn.CrossEntropyLoss()

        self.model.train()
        for ep in range(1, epochs + 1):
            total_loss = 0.0
            for xb, yb in loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                opt.zero_grad()
                logits = self.model(xb)
                loss = crit(logits, yb)
                loss.backward()
                opt.step()
                total_loss += loss.item()
            print(f"[Epoch {ep:3d}] avg loss = {total_loss / len(loader):.4f}")

# -----------------------------------------------
# 4. Main Execution
# -----------------------------------------------
if __name__ == '__main__':
    path = r"C:\Users\dell\Downloads\Weki.txt"
    seq_len = 25
    embed_dim = 50
    hidden_dim = 128

    # Step 1: Preprocess
    pre = Preprocessing(path)
    enc = pre.processing()

    # Step 2: Prepare dataset (X = seq, y = next char)
    X, y = [], []
    for i in range(len(enc) - seq_len):
        X.append(enc[i:i+seq_len])
        y.append(enc[i+seq_len])
    X = np.array(X)
    y = np.array(y)

    # Step 3: Init model
    pad_idx = 0  # No padding in char-level model but required param
    model = CustomLSTMModel(vocab_size=pre.vocab_size,
                            embed_dim=embed_dim,
                            hidden_dim=hidden_dim,
                            output_dim=pre.vocab_size,
                            pad_idx=pad_idx)

    # Step 4: Train
    learner = CustomModALWrapper(model)
    learner.fit(X[:100], y[:100], epochs=500, batch_size=64, lr=0.01)


[Epoch   1] avg loss = 3.3594
[Epoch   2] avg loss = 2.7708
[Epoch   3] avg loss = 2.5281
[Epoch   4] avg loss = 2.2354
[Epoch   5] avg loss = 1.9180
[Epoch   6] avg loss = 1.6155
[Epoch   7] avg loss = 1.4036
[Epoch   8] avg loss = 1.1601
[Epoch   9] avg loss = 0.9637
[Epoch  10] avg loss = 0.8164
[Epoch  11] avg loss = 0.6140
[Epoch  12] avg loss = 0.4879
[Epoch  13] avg loss = 0.3914
[Epoch  14] avg loss = 0.3150
[Epoch  15] avg loss = 0.2351
[Epoch  16] avg loss = 0.1775
[Epoch  17] avg loss = 0.1364
[Epoch  18] avg loss = 0.1066
[Epoch  19] avg loss = 0.0912
[Epoch  20] avg loss = 0.0705
[Epoch  21] avg loss = 0.0717
[Epoch  22] avg loss = 0.0496
[Epoch  23] avg loss = 0.0412
[Epoch  24] avg loss = 0.0380
[Epoch  25] avg loss = 0.0323
[Epoch  26] avg loss = 0.0345
[Epoch  27] avg loss = 0.0383
[Epoch  28] avg loss = 0.0257
[Epoch  29] avg loss = 0.0318
[Epoch  30] avg loss = 0.0228
[Epoch  31] avg loss = 0.0300
[Epoch  32] avg loss = 0.0203
[Epoch  33] avg loss = 0.0238
[Epoch  34

In [13]:
learner.fit(X[:100], y[:100], epochs=1000, batch_size=64, lr=0.01)

[Epoch   1] avg loss = 0.4246
[Epoch   2] avg loss = 0.0105
[Epoch   3] avg loss = 0.1696
[Epoch   4] avg loss = 0.0603
[Epoch   5] avg loss = 0.0387
[Epoch   6] avg loss = 0.0195
[Epoch   7] avg loss = 0.0141
[Epoch   8] avg loss = 0.0156
[Epoch   9] avg loss = 0.0117
[Epoch  10] avg loss = 0.0105
[Epoch  11] avg loss = 0.0081
[Epoch  12] avg loss = 0.0063
[Epoch  13] avg loss = 0.0054
[Epoch  14] avg loss = 0.0048
[Epoch  15] avg loss = 0.0038
[Epoch  16] avg loss = 0.0032
[Epoch  17] avg loss = 0.0027
[Epoch  18] avg loss = 0.0023
[Epoch  19] avg loss = 0.0021
[Epoch  20] avg loss = 0.0018
[Epoch  21] avg loss = 0.0016
[Epoch  22] avg loss = 0.0016
[Epoch  23] avg loss = 0.0015
[Epoch  24] avg loss = 0.0013
[Epoch  25] avg loss = 0.0012
[Epoch  26] avg loss = 0.0012
[Epoch  27] avg loss = 0.0011
[Epoch  28] avg loss = 0.0010
[Epoch  29] avg loss = 0.0010
[Epoch  30] avg loss = 0.0009
[Epoch  31] avg loss = 0.0009
[Epoch  32] avg loss = 0.0009
[Epoch  33] avg loss = 0.0008
[Epoch  34

In [14]:
learner.fit(X[:100], y[:100], epochs=2000, batch_size=64, lr=0.01)

[Epoch   1] avg loss = 0.0195
[Epoch   2] avg loss = 0.0262
[Epoch   3] avg loss = 0.0253
[Epoch   4] avg loss = 0.0040
[Epoch   5] avg loss = 0.0047
[Epoch   6] avg loss = 0.0048
[Epoch   7] avg loss = 0.0017
[Epoch   8] avg loss = 0.0012
[Epoch   9] avg loss = 0.0013
[Epoch  10] avg loss = 0.0009
[Epoch  11] avg loss = 0.0008
[Epoch  12] avg loss = 0.0007
[Epoch  13] avg loss = 0.0006
[Epoch  14] avg loss = 0.0005
[Epoch  15] avg loss = 0.0005
[Epoch  16] avg loss = 0.0004
[Epoch  17] avg loss = 0.0003
[Epoch  18] avg loss = 0.0003
[Epoch  19] avg loss = 0.0003
[Epoch  20] avg loss = 0.0002
[Epoch  21] avg loss = 0.0002
[Epoch  22] avg loss = 0.0002
[Epoch  23] avg loss = 0.0002
[Epoch  24] avg loss = 0.0002
[Epoch  25] avg loss = 0.0002
[Epoch  26] avg loss = 0.0001
[Epoch  27] avg loss = 0.0001
[Epoch  28] avg loss = 0.0001
[Epoch  29] avg loss = 0.0001
[Epoch  30] avg loss = 0.0001
[Epoch  31] avg loss = 0.0001
[Epoch  32] avg loss = 0.0001
[Epoch  33] avg loss = 0.0001
[Epoch  34

In [17]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


# 2. GRU Model
class GRUNetwork(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, seq_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        emb = self.embedding(x)
        out, _ = self.gru(emb)
        return self.fc(out[:, -1, :])

    def sample(self, seed_seq, length, processor, temperature=1.0):
        self.eval()
        generated = []
        window = seed_seq[-25:].copy()
        input_seq = torch.tensor(window, dtype=torch.long).unsqueeze(0)

        with torch.no_grad():
            for _ in range(length):
                emb = self.embedding(input_seq)
                out, _ = self.gru(emb)
                logits = self.fc(out[:, -1, :]) / temperature
                probs = torch.softmax(logits, dim=-1).squeeze(0)
                next_idx = torch.multinomial(probs, 1).item()
                generated.append(next_idx)
                window = window[1:] + [next_idx]
                input_seq = torch.tensor(window, dtype=torch.long).unsqueeze(0)
        return generated

# 3. Training function
def train(model, optimizer, loss_fn, X, y, epochs=100, batch_size=32, print_every=10):
    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * xb.size(0)

        avg_loss = total_loss / len(dataset)
        if epoch % print_every == 0 or epoch == 1:
            print(f"Epoch {epoch}/{epochs}, Loss: {avg_loss:.4f}")

# 4. Main Script
if __name__ == "__main__":
    path = r"C:\Users\dell\Downloads\Weki.txt"
    seq_length = 25
    embedding_dim = 50
    hidden_dim = 128
    lr = 0.01

    # Preprocess
    processor = Preprocessing(path)
    enc = processor.processing()

    # Dataset
    X_data, y_data = [], []
    for i in range(len(enc) - seq_length):
        X_data.append(enc[i:i + seq_length])
        y_data.append(enc[i + seq_length])
    X = torch.tensor(X_data, dtype=torch.long)
    y = torch.tensor(y_data, dtype=torch.long)

    # Model
    model = GRUNetwork(vocab_size=processor.vocab_size,
                       embedding_dim=embedding_dim,
                       hidden_dim=hidden_dim,
                       seq_length=seq_length)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # Train
    train(model, optimizer, loss_fn, X[:100], y[:100], epochs=500, print_every=20)




Epoch 1/500, Loss: 3.2625
Epoch 20/500, Loss: 0.0886
Epoch 40/500, Loss: 0.0246
Epoch 60/500, Loss: 0.0174
Epoch 80/500, Loss: 0.0214
Epoch 100/500, Loss: 0.0466
Epoch 120/500, Loss: 0.0077
Epoch 140/500, Loss: 0.0064
Epoch 160/500, Loss: 0.0026
Epoch 180/500, Loss: 0.0143
Epoch 200/500, Loss: 0.0015
Epoch 220/500, Loss: 0.0009
Epoch 240/500, Loss: 0.0006
Epoch 260/500, Loss: 0.0005
Epoch 280/500, Loss: 0.0004
Epoch 300/500, Loss: 0.0003
Epoch 320/500, Loss: 0.0002
Epoch 340/500, Loss: 0.0002
Epoch 360/500, Loss: 0.0002
Epoch 380/500, Loss: 0.0002
Epoch 400/500, Loss: 0.0001
Epoch 420/500, Loss: 0.0001
Epoch 440/500, Loss: 0.0001
Epoch 460/500, Loss: 0.0001
Epoch 480/500, Loss: 0.0001
Epoch 500/500, Loss: 0.0001
